# Objektumfelismerés OpenCV & TensorFlow segítségével

Készítette: Gellén Rebeka, J97N2Y, 2022/11/_

#### A TensorFlow-ról

A TensorFlow megkönnyíti a modellek betanítását, alkalmazását. Szövegfeldolgozás órán ismerkedtem meg vele, ott szöveggenerálás folyamán használtuk. Kíváncsi voltam, képfeldolgozás során mennyire használható.

TFLite formátumú modellel fogunk dolgozni, ugyanis ez méretben kisebb és jobb teljesítményű, mint a nagyobb modellek.

Felhasznált modell: https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1

Ez a modell képes: 

1. Bekeretezni a detektált elemet.
2. Megadja, mennyi az esélye egy adott osztálynak
3. Felismeri az osztályt
4. Megadja a detektálások számát

!! Ahhoz, hogy az opencv, tensorflow, python és a felhasznált modellünk működni tudjon, elég sok mindent kell telepíteni és összekötni, így nem érdemes csak úgy nekiugrani, mert nem két perc munka. !!

In [1]:
#!pip install tensorflow

In [2]:
#!pip install tensorflow_hub

In [3]:
#!pip install opencv-python

In [4]:
#!pip install numpy

In [5]:
#!pip install pandas

In [6]:
#!pip install matplotlib

## Objektumfelismerés állóképen

Első lépésben betöltjük a modellt és a beolvasott képet fogjuk olyanra formázni, hogy a tensorflow dolgozni tudjon vele.

In [7]:
import tensorflow_hub as hub
import cv2
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# A modelt direkt a weboldalról töltjük be, de le is lehet tölteni a számítógépre
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")

In [ ]:
width = 1028
height = 1028

#beolvassuk a képet
img = cv2.imread('aaaa.jpg')
#méretezzük
inp = cv2.resize(img, (width , height ))

#RGB-be konvertáljuk a képet
rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

#uint8-ba konvertálunk
rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

#az rgb_tensor dimenzióit kibővítjük
rgb_tensor = tf.expand_dims(rgb_tensor , 0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(rgb)

Importáljuk a címkéket

In [ ]:
boxes, scores, classes, num_detections = detector(rgb_tensor)

In [ ]:
# Címkék és osztályok betöltése
labels = pd.read_csv('labels.csv',sep=';',index_col='ID')
labels = labels['OBJECT (2017 REL.)']

In [ ]:
labels.head()

Prediktálás

In [ ]:
# Prediktálás
pred_labels = classes.numpy().astype('int')[0] 
pred_labels = [labels[i] for i in pred_labels]
pred_boxes = boxes.numpy()[0].astype('int')
pred_scores = scores.numpy()[0]

In [ ]:
# Keretezés és címkézés a képen
for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < 0.5:
            continue
            
        score_txt = f'{100 * round(score)}%'
        img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),2)      
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_boxes, label,(xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
        cv2.putText(img_boxes,score_txt,(xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_boxes)